# Single Cell Simulation Example with OBI-One Form Logic

This notebook demonstrates how to run a single cell simulation using the OBI-One form-based workflow, similar to the circuit simulation example.

In [ ]:
from entitysdk import Client
from obi_auth import get_token
from pathlib import Path

from obi_notebook import get_projects
from obi_notebook import get_entities
import obi_one as obi

token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

In [ ]:
client = Client(environment="production", project_context=project_context, token_manager=token)

# Optional: Download using unique ID
entity_ID = "<MEMODEL-ID>"  # <<< FILL IN UNIQUE MEModel ID HERE

if entity_ID != "<MEMODEL-ID>":
    mmemodel_ids = [entity_ID]
else:
# Alternative: Select from a table of entities
    mmemodel_ids = []
    mmemodel_ids = get_entities.get_entities("memodel", token, mmemodel_ids,
                                            project_context=project_context,
                                            multi_select=False,
                                            default_scale="small")

In [ ]:
sonata_dir=Path('./sonata')
memodel = obi.MEModelFromID(id_str=str(mmemodel_ids[0]))
circuit = memodel.stage_as_form_circuit(db_client=client, dest_dir=Path(sonata_dir))

In [6]:
import obi_one as obi
from pathlib import Path

# === Parameters ===
sim_duration = 1000.0  # ms
cell_id = 0  # Change as needed

# === 1. Build Form (SimulationsForm) ===
sim_form = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(
    campaign_name="Single Cell Simulation",
    campaign_description="Simulation of a single cell with constant current stimulus"
)
sim_form.set(info, name="info")

# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

# Stimulus
stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps.ref, duration=800.0, amplitude=0.5
)
sim_form.add(stimulus, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording()
sim_form.add(recording, name="SomaVoltage")


init = obi.SimulationsForm.Initialize(
    circuit=circuit,
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")

In [10]:
# === 2. Wrap into a Simulation ===
simulation = obi.Simulation(**sim_form.model_dump())
simulation.coordinate_output_root = "."

# === 3. Generate simulation_config.json + node_sets.json ===
simulation.generate()  # Add db_client if needed

In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {sonata_dir}/mechanisms

In [ ]:
# === 4. Run the simulation (BlueCelluLab backend) ===
from obi_one.scientific.simulation.execution import run

run(
    simulation_config='simulation_config.json',
    simulator='bluecellulab',
    save_nwb=False
)

## Results
The results are stored in the `output` directory. You can analyze the voltage traces and other outputs as needed.

In [13]:
import bluepysnap
simulation_config_path = Path(".") / "simulation_config.json"
snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

0 1000.0 0.025
['All']


In [14]:
population_name = circuit.default_population_name

spike_pop = spikes[population_name]
node_population = spike_pop.nodes
filtered = spikes.filter( t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

,ids,population
times,,
1.000000e-10,0,All
1.570000e+01,0,All
3.242500e+01,0,All
4.920000e+01,0,All
6.595000e+01,0,All


In [15]:
snap_simulation.reports

{'SomaVoltage': <bluepysnap.frame_report.SomaReport at 0x34230fb90>}

In [16]:
soma_report = snap_simulation.reports['SomaVoltage']
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt
)  # Gives a warning in case the dt differs from simulation.dt

0.0 1000.0 0.1
